<a href="https://colab.research.google.com/github/avnishks/colab/blob/main/Exploratory_Data_Analysis_Cleveland_Heart_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shapely as shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as ss
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

In [3]:
df = pd.read_csv('processed.cleveland.data', sep=",", header=None)
df.columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach",
              "exang", "oldpeak", "slope", "ca", "thal", "diseaseStat"]
print("df_shape: ", df.shape)

df_shape:  (303, 14)


In [4]:
df.dtypes

age            float64
sex            float64
cp             float64
trestbps       float64
chol           float64
fbs            float64
restecg        float64
thalach        float64
exang          float64
oldpeak        float64
slope          float64
ca              object
thal            object
diseaseStat      int64
dtype: object

In [1]:
print("rows with missing 'ca' values: ",
      df[pd.to_numeric(df.ca, errors='coerce').isnull()][["ca", "thal"]], sep='\n')
print('\n')
print("rows with missing 'thal' values: ",
      df[pd.to_numeric(df.thal, errors='coerce').isnull()][["ca", "thal"]], sep='\n')